# Script to Create User Profiles based on Past Item Interactions - [GOOGLE COLAB]

In [1]:
# Install missing libraries
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.6 MB/s eta 0:00:00


In [2]:
#@title Import Libraries
import os
import pickle
import numpy as np
import random
import itertools
import io
import json

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `MIDS-Capstone` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `MIDS

In [5]:
# Function to download the data - AWS
def load_dict_from_s3(bucket_name, file_key):
    """
    Load dictionary from S3 pickle file.
    """
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    file_content = response['Body'].read()
    data_dict = pickle.loads(file_content)

    return data_dict

In [6]:
# Function to download the data - Google Colab
def load_dict_from_drive(file_path):
    """
    Load dictionary from Google Drive pickle file.
    """
    with open(file_path, 'rb') as file:
        data_dict = pickle.load(file)

    return data_dict

## Define LLM Task to create User Profiles

In [ ]:
# AWS -- OUDATED
class ChatBot:
    def __init__(self, model_name="meta-llama-3-1-8b-instruct-005734"):
        # Initialize the SageMaker runtime client
        self.sagemaker_runtime = boto3.client('sagemaker-runtime')
        self.endpoint_name = "jumpstart-dft-llama-3-1-8b-instruct-20250225-005734"
        self.model_name = model_name

    def get_response(self, message):
        try:
            payload = json.dumps({"inputs": message})
            response = self.sagemaker_runtime.invoke_endpoint(
                EndpointName=self.endpoint_name,
                ContentType='application/json',
                Body=payload
            )
            result = json.loads(response['Body'].read().decode())

            # Print raw response for debugging
            print("\n===== RAW RESPONSE FROM SAGEMAKER =====")
            print(result)
            print("========================================\n")

            # Extract and clean response
            if isinstance(result, list) and len(result) > 0:
                generated_text = result[0].get('generated_text', '').strip()
            else:
                generated_text = "Error: Unexpected response format from SageMaker."

            return generated_text

        except Exception as e:
            print(f"Error in get_response: {e}")
            return "Error: Unable to generate a response."

    def build_prompt(self, item_list, item_info_dict):
        """
        Build a user profiling prompt based on books read
        """
        content_begin = """
        Now, you are a book analyst tasked to identify a user's favorite book genres based on their past readings. I will provide you with a list of books that a user has watched in the past.
        Each book contains two pieces of information: title and genre. Based on this information, identify in the provided data the user's TOP 3 most-read genres.

        Here are some examples of what the output should look like:

        ### Example 1:
        **INPUT**
        User's Reading History:
        - Title: The Great Gatsby, Genre: Fiction
        - Title: Thinking, Fast and Slow, Genre: Psychology
        - Title: 1984, Genre: Dystopian
        - Title: Brave New World, Genre: Dystopian
        - Title: Fahrenheit 451, Genre: Dystopian
        - Title: Blink, Genre: Psychology
        - Title: The Catcher in the Rye, Genre: Fiction

        **OUTPUT**
        User favorite genres: [Dystopian], [Fiction], [Psychology]

        ---

        ### Here is the list of previously read books:
        """

        content_end = """
        Please provide the profile strictly in the following format:
        User interests: [Interest 1], [Interest 2], [Interest 3]

        Emphasize that only the most likely three genres should be provided, and strictly adhere to the above format.
        """

        # Get Data and remove dups
        unique_items = set()
        filtered_list = []

        for item_id in item_list:
            title = item_info_dict.get(item_id, {}).get('standard_title', 'Unknown Title')
            genre = item_info_dict.get(item_id, {}).get('parsed_categories', 'Unknown Genre')

            if isinstance(genre, list):
                genre = ", ".join(genre)

            item_tuple = (title, genre)
            if item_tuple not in unique_items:
                unique_items.add(item_tuple)
                filtered_list.append(f"- Title: {title}, Genre: {genre}")

        # Join unique filtered items
        content_item_list = "\n".join(filtered_list)

        return content_begin + content_item_list + content_end


    def chat(self, item_list, item_info_dict, user_Id):
        """
        Generate a user profile based on historical book interactions.
        """
        content = self.build_prompt(item_list, item_info_dict)

        # Print the raw prompt before sending it to the LLM
        print("\n===== RAW INPUT TO LLM =====")
        print(content)
        print("============================\n")

        response = self.get_response(content)

        # Print the response received from the model
        print("\n===== RAW OUTPUT FROM LLM =====")
        print(response)
        print("===============================\n")

        return {user_Id: response}

In [36]:
# Google Colab
class ChatBot:
    def __init__(self, model_name="meta-llama/Llama-3.1-8B-Instruct"):
        """
        Initialize the ChatBot with the specified model name.
        """
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token

        # Optimize model loading with 4-bit quantization
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16
        )

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config
        )

    def get_response(self, message):
        try:
          formatted_prompt = f"[INST] {message} [/INST]"
          inputs = self.tokenizer(
              formatted_prompt,
              return_tensors="pt",
              padding=True,
              truncation=True,
              max_length=1024
            ).to("cuda")

          attention_mask = inputs.attention_mask

          output = self.model.generate(
              inputs.input_ids,
              attention_mask=attention_mask,
              max_new_tokens=256,
              temperature=0.7,
              top_p=0.9,
              do_sample=True,
              pad_token_id=self.tokenizer.eos_token_id,
              )

          response = self.tokenizer.decode(output[0], skip_special_tokens=True)
          return response[len(formatted_prompt):].strip()

        except Exception as e:
          print(f"Error in get_response: {e}")
          return "Error: Unable to generate a response."

    def chat(self, user_item_dict, item_info_dict, user_id, result_dict, user_preferences, recommended_item, user_feedback):
        """
        Explain why recommendations were generated for a user

        Args:
            user_item_dict (dict): User-item interaction history.
            item_info_dict (dict): Book metadata (category, title, etc.).
            user_id (str): Target user ID.
            recommended_item (str): The recommended book ID.
            user_preferences (dict): Dictionary of user preferences {user_id: [categories]}.
            user_feedback (dict): Dictionary of user feedback {user_id: {"liked": [book_ids], "disliked": [book_ids]}}.

        Returns:
            str: Explanation generated by the LLM
        """

        # System Instructions
        content_begin = (
            "[INST] <<SYS>>\n"
            "You are a recommendation assistant that explains why books were suggested to a user.\n"
            "Your reasoning should be based on:\n"
            "- The user's **past interactions** (books they have read before).\n"
            "- The user's **onboarding preferences** (favorite genres and categories).\n"
            "- The recommended books (collaborative filtering system output).\n"
            "- The user's **feedback** (liked/disliked books).\n"
            "If a recommended book belongs to a **new category** the user has never interacted with, assume it was suggested "
            "based on preferences of similar users.\n"
            "<</SYS>>\n\n"
        )


        # 1. User's historical interactions
        content_user_history = f"### User {user_id}'s Historical Interactions:\n"
        user_categories = set()

        for item_id in user_item_dict.get(user_id, []):
            book_info = item_info_dict.get(item_id, {})
            title = book_info.get('standard_title', 'Unknown Title')
            categories = book_info.get('parsed_categories', [])

            # Ensure is a list
            if isinstance(categories, str):
                categories = [categories]

            user_categories.update(categories)
            content_user_history += f"- **{title}** (Genres: {', '.join(categories)})\n"

        # 2. User's onboarding preferences
        user_pref_categories = set(user_preferences.get(user_id, []))
        content_user_prefs = f"\n### User {user_id}'s Onboarding Preferences:\n"
        content_user_prefs += f"- Preferred Categories: {', '.join(user_pref_categories)}\n"

        # 3. Recommended book (Known vs. New Category)
        book_info = item_info_dict.get(recommended_item, {})
        title = book_info.get('standard_title', 'Unknown Title')
        categories = book_info.get('parsed_categories', [])

        if isinstance(categories, str):
            categories = [categories]

        # Determine recommendation type
        if user_categories.intersection(categories) or user_pref_categories.intersection(categories):
            recommendation_reason = "based on the user's past interactions and interests."
        else:
            recommendation_reason = "suggested based on similar users' preferences."

        content_recommendation = (
            f"\n### Recommended Book: **{title}**\n"
            f"- Genre(s): {', '.join(categories)}\n"
            f"- Reason: {recommendation_reason}\n"
        )

        # 4. User feedback (liked/disliked books)
        content_feedback = f"\n### User {user_id}'s Feedback on Past Recommendations:\n"
        liked_books = [item_info_dict.get(item_id, {}).get('standard_title', 'Unknown Title') for item_id in user_feedback.get(user_id, {}).get("liked", [])]
        disliked_books = [item_info_dict.get(item_id, {}).get('standard_title', 'Unknown Title') for item_id in user_feedback.get(user_id, {}).get("disliked", [])]

        if liked_books:
            content_feedback += f"- **Liked Books**: {', '.join(liked_books)}\n"
        if disliked_books:
            content_feedback += f"- **Disliked Books**: {', '.join(disliked_books)}\n"
        if not liked_books and not disliked_books:
            content_feedback += "- No feedback recorded.\n"

        # 5. Explanation request
        content_request = (
            "\n### Explanation Task:\n"
            "Using the data above, explain why the recommended book was suggested. "
            "For books that match the user’s past behavior, reinforce the alignment. "
            "For books from new categories, mention that they were suggested based on similar users' preferences.\n\n"
            "Format your response as follows:\n"
            "- **[Book Title]**: Explanation of why it was recommended.\n\n"
            "[/INST]"
        )

        # Combine all parts
        content = content_begin + content_user_history + content_user_prefs + content_recommendation + content_feedback + content_request

        print("SEND: ")
        print(content)

        # Get response
        response = self.get_response(content)
        result_dict[user_id] = response

        print("GPT: ")
        print(response)
        print()

## Upload the Data Sources

In [ ]:
# Setup Paths - AWS
s3 = boto3.client('s3')
bucket_name = 'w210recsys'

### User-Item Historical Interactions

In [ ]:
# #Import User-Item interactions dictionary - AWS
# file_key = 'processed/user_item_dict.pkl'
# user_item_dict = load_dict_from_drive(bucket_name, file_key)

In [9]:
# Import User-Item interactions dictionary - Colab
file_key = '/content/drive/My Drive/Capstone - Spring 2025 Personal/user_profiles/user_item_dict.pkl'
user_item_dict = load_dict_from_drive(file_key)

In [10]:
for i, (key, value) in enumerate(user_item_dict.items()):
    if i >= 5:
        break
    print(f"User ID: {key}, Items: {value}")

User ID: A01038432MVI9JXYTTK5T, Items: [15693, 17623, 20231, 23671, 25313, 26282, 34776, 34901, 51220, 55669, 56795, 60236]
User ID: A100NGGXRQF0AQ, Items: [141, 3330, 5426, 8736, 15511, 21557, 25655, 26605, 38825, 45116, 59372, 60614, 63574, 65047, 70451]
User ID: A100V1W0C8BWOL, Items: [21901, 24501, 28429, 29020, 30221, 32803, 36113, 10022, 47988, 58373, 64427]
User ID: A100YHBWL4TR4D, Items: [7209, 7396, 11726, 14023, 19355, 21738, 27003, 30061, 33300, 45369, 47419, 45667, 50844, 68750]
User ID: A101446I5AWY0Z, Items: [2219, 2269, 3322, 4343, 5987, 8056, 10827, 15499, 16602, 17109, 23670, 31893, 32068, 32259, 32596, 39109, 40873, 41154, 48451, 51637, 55990, 66303, 67211, 68516, 72230]


In [11]:
###### FOR TESTING ONLY ######
N = 1
sampled_keys = random.sample(list(user_item_dict.keys()), N)
sampled_dict = {key: user_item_dict[key] for key in sampled_keys}

print(sampled_dict)

{'A1DI9POOHWJTYA': [2183, 3986, 12346, 32785, 38824, 39615, 46918, 59047, 60219, 63388, 69533]}


### Books Metadata

In [12]:
# #Import Items (books) metadata dictionary - AWS
# file_key = 'processed/item_info_dict.pkl'
# item_info_dict = load_dict_from_s3(bucket_name, file_key)

In [12]:
# Import Items (books) metadata dictionary - Colab
file_key ='/content/drive/My Drive/Capstone - Spring 2025 Personal/user_profiles/item_info_dict.pkl'
item_info_dict = load_dict_from_drive(file_key)

In [13]:
for key, value in itertools.islice(item_info_dict.items(), 5):
    print(key, ":", value)

0 : {'standard_title': 'dr seuss american icon', 'parsed_categories': ['Biography & Autobiography']}
1 : {'standard_title': 'wonderful worship in smaller churches', 'parsed_categories': ['Religion']}
2 : {'standard_title': 'rising sons and daughters life among japans new young', 'parsed_categories': ['Social Science']}
3 : {'standard_title': 'muslim womens choices religious belief and social reality', 'parsed_categories': ['Religion']}
4 : {'standard_title': 'dramatica for screenwriters', 'parsed_categories': ['Reference']}


### User's Onboarding Input

In [14]:
file_key = '/content/drive/My Drive/Capstone - Spring 2025 Personal/user_profiles/onboarding_dict.pkl'
onboarding_dict = load_dict_from_drive(file_key)

In [15]:
for key, value in itertools.islice(onboarding_dict.items(), 5):
    print(key, ":", value)

A01038432MVI9JXYTTK5T : ['American fiction', 'POETRY', 'Materia medica', 'Medieval']
A100NGGXRQF0AQ : ['Fruit', 'French drama']
A100V1W0C8BWOL : ['Athens (Greece)', 'Imagination', 'ices', 'Cross-country (Horsemanship)', 'Coming of age']
A100YHBWL4TR4D : ['Israel', 'Nathaniel']
A101446I5AWY0Z : ['Kuṇḍalinī']


### User's Feedback Data

In [16]:
file_key = '/content/drive/My Drive/Capstone - Spring 2025 Personal/user_profiles/feedback_dict.pkl'
feedback_dict = load_dict_from_drive(file_key)

In [17]:
for key, value in itertools.islice(feedback_dict.items(), 5):
    print(key, ":", value)

A01038432MVI9JXYTTK5T : {'liked': [], 'disliked': [25313]}
A100NGGXRQF0AQ : {'liked': [3330], 'disliked': [141]}
A100V1W0C8BWOL : {'liked': [], 'disliked': [30221, 10022, 58373, 64427]}
A100YHBWL4TR4D : {'liked': [7209, 11726, 47419], 'disliked': [27003]}
A101446I5AWY0Z : {'liked': [], 'disliked': []}


### Recommendations

In [18]:
file_key = '/content/drive/My Drive/Capstone - Spring 2025 Personal/user_profiles/cf_dict.pkl'
cf_dict = load_dict_from_drive(file_key)

In [19]:
for key, value in itertools.islice(cf_dict.items(), 5):
    print(key, ":", value)

A01038432MVI9JXYTTK5T : [70013, 65089, 48129, 48178, 33184, 56203]
A100NGGXRQF0AQ : [39119, 38995, 38952, 68863, 2071, 40865]
A100V1W0C8BWOL : [54344, 18725, 33473, 49862]
A100YHBWL4TR4D : [57027, 17324, 67786]
A101446I5AWY0Z : [9830, 69737, 24746, 5877]


### Load the LLM model

In [37]:
# Pick a model
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [38]:
# Create an instance of the ChatBot class
chatbot = ChatBot(model_name=model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
print(f"Number of unique users: {len(user_item_dict)}")
print(f"Number of unique books: {len(item_info_dict)}")

Number of unique users: 14402
Number of unique books: 72308


In [39]:
# Create dictionary to store results
result_dict = {}
counter = 0

In [40]:
# Loop through sampled users and get explanations
for user_id in sampled_dict:
    recommended_books = cf_dict.get(user_id, [])  # Get recommendations for user

    result_dict[user_id] = []

    for book_id in recommended_books:
        # Get response
        response = chatbot.chat(
            user_item_dict,        # User-item interactions
            item_info_dict,        # Book metadata
            user_id,               # Current user
            {},                    # Empty result_dict
            onboarding_dict,       # User preferences
            book_id,               # Recommended book ID
            feedback_dict          # User feedback
        )

        # Store responses
        result_dict[user_id].append({book_id: response})

        # Print response
        print(f"User {user_id}, Book {book_id} Response: {response}")

        # increment and print counter
        counter += 1
        print(counter)

# Final counter
print("Total Books Processed:", counter)

SEND: 
[INST] <<SYS>>
You are a recommendation assistant that explains why books were suggested to a user.
Your reasoning should be based on:
- The user's **past interactions** (books they have read before).
- The user's **onboarding preferences** (favorite genres and categories).
- The recommended books (collaborative filtering system output).
- The user's **feedback** (liked/disliked books).
If a recommended book belongs to a **new category** the user has never interacted with, assume it was suggested based on preferences of similar users.
<</SYS>>

### User A1DI9POOHWJTYA's Historical Interactions:
- **eragon inheritance, book one** (Genres: Dragons)
- **chinese** (Genres: Family & Relationships)
- **the lord of the rings - boxed set** (Genres: Young Adult Fiction)
- **the lord of the rings** (Genres: Baggins, Bilbo (Fictitious character))
- **the lord of the rings trilogy** (Genres: Fiction)
- **the zero stone** (Genres: Fiction)
- **dreams of terror and death the dream cycle of h 

KeyboardInterrupt: 

In [39]:
result_dict

{'A3T46YOGTFC5IG': None}

### NEXT STEPS